# Build your own visualization and add it to your website :)

Go to [DataWrapper](https://app.datawrapper.de/) and create your account to build your first visualization.

![4dsd](https://raw.githubusercontent.com/diazcelsa/tech_for_good/master/img/datawrapper.png)

## We will use Python to access and transform the data for the visualization

In [0]:
# Make python package imports
import pandas as pd

# Import when using the notebook from google drive Colab
from google.colab import files

# Hide warnings
import warnings
warnings.filterwarnings('ignore')

In [0]:
### Define functions for data extraction

# Filter out the datasets that do not have access throught the API or geolocation info
def filter_datasets_by_criteria(data, geo_location=True):
    
    # Selection of datasets that have geolocation info and access through the API
    if geo_location:
        condition_geolocation = (data['geolocation'] != 'No')
        condition_api = (data['api'] == 'Yes')
        data_ = data[condition_geolocation & condition_api]
    
    # Selection of datasets that have access through the API
    else:
        condition_api = (data['api'] == 'Yes')
        data_ = data[condition_geolocation & condition_api]

    return data_

  
# Get the info about all datasets in the city hall of Barcelona site  
def get_geolocated_barcelona_datasets(url):
    
    # Request information about all datasets available
    response = pd.read_json(url)
    response_details = pd.DataFrame.from_records(response.loc['results','result'])
    
    # Extract only datasets that do not have geolocation and access through the API 
    response_details_clean = filter_datasets_by_criteria(response_details, geo_location=True)
    
    # Extract names of datasets
    dataset_name_list = response_details_clean['code'].tolist()
    
    return dataset_name_list, response_details_clean

  
# Call to the city hall API to access the data for the given dataset
def get_dataset(data, name="sanitat serveis sanitaris"):
    
    # Extract the dataset for the given name 
    dataset_url = data[data['code'].str.contains(name)]['resources'].values[0][0]['url']
    
    # Download the requested dataset
    return pd.read_csv(dataset_url)
  
def get_number_of_business_per_neighborhoods(data, list_businesses):
  
  # Select the data for the types of business of interest
  data_selection = data[data['N_ACT'].isin(list_businesses)]
  data_selection = data_selection[['LONGITUD','LATITUD','N_ACT','N_LOCAL','Nom_Barri','Codi_Barri']]
  
  # Calculate the number of businesses per type and neighborhoods and the geographical center of mass of all them in each neighborhood
  center_locations = data_selection[['Codi_Barri','LONGITUD','LATITUD','N_ACT']].groupby(['N_ACT', 'Codi_Barri'], as_index=False).mean()
  n_businesses = data_selection[['Codi_Barri','Nom_Barri','LONGITUD','N_ACT']].groupby(['N_ACT', 'Nom_Barri', 'Codi_Barri'], as_index=False).count()
  n_businesses = n_businesses.rename(columns={'LONGITUD':'n_events'})
  
  # Combine center of businesses per neighborhood and number of businesses for the visualization
  biz_neighborhood_info = center_locations.merge(n_businesses, left_on=['N_ACT','Codi_Barri'], right_on=['N_ACT','Codi_Barri'])
  
  return biz_neighborhood_info

## Get the list of public datasets provided by the City Hall of Barcelona

![sdfs](https://raw.githubusercontent.com/diazcelsa/tech_for_good/master/img/open_data_bcn.png)

In [0]:
# Get info from the datasets available in the city hall of Barcelona
url="https://opendata-ajuntament.barcelona.cat/data/api/3/action/package_search?rows=10000"
dataset_name_list, response_details_clean = get_geolocated_barcelona_datasets(url)

In [0]:
dataset_name_list

['educacio ensenyament infantil',
 'educacio ensenyament no reglat',
 'sanitat serveis sanitaris',
 'sanitat hospitals atencio primaria',
 'sanitat farmacies',
 'FONTS_ORNAMENTALS',
 'equipaments culturals icub',
 'barcelona-activa-subvencions',
 'ServeisSocials_MenjadorsSocials',
 'serveissocials residenciesgentgran',
 'serveissocials centresdiagentgran',
 'serveissocials centresserveissocials',
 'serveissocials-oficinesbenestarsocial',
 'serveissocials habitatgestutelats',
 'serveissocials puntsassessoramentenergetic',
 'serveissocials equipsatencioinfanciaadolescencia',
 'serveissocials puntsinformacioiatenciodones',
 'aforaments descriptiu',
 'qualitat_aire_estacions_BCN',
 'dades xarxa biblioteques catalunya',
 'Arbrat zona',
 'trams aparcament superficie',
 'punts critics neteja barcelona',
 'COBERTURA_VEGETAL_NDVI',
 'IRIS',
 'HABITATGES_US_TURISTIC',
 'dades ateneus',
 'dades projectes fabriques creacio',
 'ACCIDENTS_CAUSES_GU_BCN',
 'ACCIDENTS_PERSONES_GU_BCN',
 'ACCIDENTS_TIP

# 1. How is the density of a given type of business over the neighborhoods in Barcelona?

![ss](https://raw.githubusercontent.com/diazcelsa/tech_for_good/master/img/handcraft_biz_barna.png) ![sss](https://raw.githubusercontent.com/diazcelsa/tech_for_good/master/img/tourism_biz_barna.png)

In [0]:
# Get the dataset from comercial activities per type of activity
name_dataset = "CENS_ACTIVITATS_COMERCIALS"
dataset = get_dataset(response_details_clean, name=name_dataset)

In [0]:
dataset.head(2)

,ID_BCN,ID_PRINCIP,N_PRINCIP,ID_SECTOR,N_SECTOR,ID_GRUPACT,N_GRUPACT,ID_ACT,N_ACT,N_LOCAL,SN_CARRER,SN_MERCAT,ID_MERCAT,N_MERCAT,SN_GALERIA,N_GALERIA,SN_CCOMERC,ID_CCOMERC,N_CCOMERC,N_CARRER,NUM_POLICI,REF_CAD,DATA,Codi_Barri,Nom_Barri,Codi_Districte,N_DISTRI,N_EIX,SN_EIX,SEC_CENS,Y_UTM_ETRS,X_UTM_ETRS,LATITUD,LONGITUD
0,49,1,Actiu,1,Comerç al detall,4,Equipament personal,4003,Vestir,INSIDE,0,0,0,,0,,1,CC003,Maremàgnum,MOLL ESPANYA,5.0,1813205DF3811D,2016-06-07,2,el Barri Gòtic,1,Ciutat Vella,NaN,0,1030,4580750.861,431675.395,41.375377,2.182944
1,50,1,Actiu,2,Serveis,14,"Restaurants, bars i hotels (Inclòs hostals, pe...",14000,Serveis de menjar i begudes,KURTZ & GUT,0,0,0,,0,,1,CC003,Maremàgnum,MOLL ESPANYA,5.0,1813205DF3811D,2016-06-07,2,el Barri Gòtic,1,Ciutat Vella,NaN,0,1030,4580752.167,431687.786,41.375390,2.183092


#### How does each of the business events looks like?

In [0]:
dataset.iloc[0]

ID_BCN                             49
ID_PRINCIP                          1
N_PRINCIP                       Actiu
ID_SECTOR                           1
N_SECTOR             Comerç al detall
ID_GRUPACT                          4
N_GRUPACT         Equipament personal
ID_ACT                           4003
N_ACT                          Vestir
N_LOCAL                        INSIDE
SN_CARRER                           0
SN_MERCAT                           0
ID_MERCAT                           0
N_MERCAT                             
SN_GALERIA                          0
N_GALERIA                            
SN_CCOMERC                          1
ID_CCOMERC                      CC003
N_CCOMERC                  Maremàgnum
N_CARRER                 MOLL ESPANYA
NUM_POLICI                          5
REF_CAD                1813205DF3811D
DATA                       2016-06-07
Codi_Barri                          2
Nom_Barri              el Barri Gòtic
Codi_Districte                      1
N_DISTRI    

#### How many businesses do we have available?

In [0]:
len(dataset)

78033

**Take into account**
* The geocoding information is at district, neighborhood and latitude/longitude level
* There are almost 80k businesses so we will aggregate the number of businesses per type and neighborhood

#### How many types of business are described?

In [0]:
dataset['N_ACT'].unique()

array(['Vestir', 'Serveis de menjar i begudes', 'Calçat i pell',
       'Drogueria i perfumeria', 'Joieria, rellotgeria i bijuteria',
       "Centres d'estètica", 'Begudes', 'Optiques i fotografia',
       'Informàtica', 'Resta alimentació',
       'Equipaments culturals i recreatius', 'Tintoreries',
       'Mobles i articles fusta i metall', 'Material equipament llar',
       'Joguines i esports', 'Souvenirs i basars', 'Altres',
       'Perruqueries', 'Pa, pastisseria i làctics', 'Carn i Porc',
       'Llibres, diaris i revistes', 'Grans magatzems i hipermercats',
       'Peix i marisc', 'Herbolaris', 'Finances i assegurances',
       'Esports', 'Arranjaments', 'Ous i aus', 'Merceria',
       'Activitats de transport i emmagatzematge', '-', 'Farmàcies',
       'Floristeries', 'Veterinaris / Mascotes', 'Associacions',
       "Serveis d'allotjament", 'Aparells domèstics',
       'Manteniment, neteja i similars',
       'Reparacions (Electrodomèstics i automòbils)',
       'Agències de v

#### Select the types of business to visualize in the map of Barcelona

In [0]:
businesses_handcraft = ['Fabricació tèxtil', 'Mobles i articles fusta i metall', 'Reparacions (Electrodomèstics i automòbils)']
businesses_tourism = ['Souvenirs i basars', "Serveis d'allotjament", 'Agències de viatge']

# Aggregate the data per neighborhood for the given businesses
biz_neighborhood_info = get_number_of_business_per_neighborhoods(dataset, businesses_tourism)

In [0]:
biz_neighborhood_info.head()

,N_ACT,Codi_Barri,LONGITUD,LATITUD,Nom_Barri,n_events
0,Agències de viatge,1,2.168302,41.379526,el Raval,22
1,Agències de viatge,2,2.176476,41.385064,el Barri Gòtic,2
2,Agències de viatge,3,2.191504,41.380565,la Barceloneta,1
3,Agències de viatge,4,2.178289,41.385539,"Sant Pere, Santa Caterina i la Ribera",3
4,Agències de viatge,5,2.179568,41.395398,el Fort Pienc,6


In [0]:
# Download the dataset for visualizations
biz_neighborhood_info.to_csv('comercios_barcelona.csv',index=False, encoding='utf-8')
files.download('comercios_barcelona.csv')

#### Build your own visualization in https://app.datawrapper.de

# 2. How are distributed the subsidies from Barcelona Activa distributed over the city?

![sde](https://raw.githubusercontent.com/diazcelsa/tech_for_good/master/img/barcelona_activa.png)

In [0]:
# Select the dataset about the barcelona activa subsidies
name_dataset = "barcelona-activa-subvencions"
dataset = get_dataset(response_details_clean, name=name_dataset)

In [0]:
dataset.head(2)

,Nom_projecte,Breu_Descripció_Projecte,Entitat_Sol·licitant,Desc_Modalitat,Col.lectiu_Destinatari,Adreça,Codi_Postal,Ciutat,Longitud,Latitud,Àmbit_Territorial
0,Beques per a Joves estudiants de Trinitat Nova,Reconèixer i donar suport als joves en els seu...,ASSOCIACIÓ DEL COMERÇ I DE LES EMPRESES DE TRI...,Emprenedoria i empresa de proximitat,"Joves estudiants majors d’edat, empadronats al...","Carrer de la Fosca, 10",8033,Barcelona,2.18409,41.44801,LA TRINITAT NOVA
1,Servei d'activació laboral per dones amb disca...,Millorar la situació de pobresa de trenta done...,ASSOCIACIÓ DE DONES NO ESTÀNDARDS (DONNES),Foment de l'ocupació de qualitat al territori,"Dones amb discapacitat física i/o sensorial, a...","Carrer de Sant Antoni Maria Claret, 31, entresol",8025,Barcelona,2.16544,41.40473,BARCELONA


#### How does each of the subsidies events looks like?

In [0]:
dataset.iloc[0]

Nom_projecte                    Beques per a Joves estudiants de Trinitat Nova
Breu_Descripció_Projecte     Reconèixer i donar suport als joves en els seu...
Entitat_Sol·licitant         ASSOCIACIÓ DEL COMERÇ I DE LES EMPRESES DE TRI...
Desc_Modalitat                            Emprenedoria i empresa de proximitat
Col.lectiu_Destinatari       Joves estudiants majors d’edat, empadronats al...
Adreça                                                  Carrer de la Fosca, 10
Codi_Postal                                                               8033
Ciutat                                                               Barcelona
Longitud                                                               2.18409
Latitud                                                                 41.448
Àmbit_Territorial                                             LA TRINITAT NOVA
Name: 0, dtype: object

#### How many subsidies do we have available?

In [0]:
len(dataset)

97

**Take into account**
* The geocoding information is at zip code and latitude/longitude level
* There are almost 100 subsidies so there is no need to aggregate events per zip code

#### How many types of subsidies are described?

In [0]:
dataset['Desc_Modalitat'].unique()

array(['Emprenedoria i empresa de proximitat',
       "Foment de l'ocupació de qualitat al territori",
       'Economies comunitàries i innovació social', 'Turisme responsable',
       "Instal·lació d'activitat econòmica en plantes baixes buides",
       'Innovació Social Digital als territoris'], dtype=object)

#### Select relevant information and save it for the final visualization

In [0]:
dataset.columns

Index(['Nom_projecte ', 'Breu_Descripció_Projecte ', 'Entitat_Sol·licitant ',
       'Desc_Modalitat', 'Col.lectiu_Destinatari ', 'Adreça ', 'Codi_Postal ',
       'Ciutat', 'Longitud', 'Latitud', 'Àmbit_Territorial '],
      dtype='object')

In [0]:
dataset_selection = dataset[['Nom_projecte ', 'Desc_Modalitat', 'Longitud', 'Latitud']]

# Download the dataset for visualizations
dataset_selection.to_csv('barcelona_activa.csv',index=False, encoding='utf-8')
files.download('barcelona_activa.csv')

#### Build your own visualization in https://app.datawrapper.de

# Add the visualization to your website!

Go to the [this Github repository](https://github.com/diazcelsa/tech_for_good) and download the repository.